In [1]:
import numpy as np
import pandas as pd 


In [2]:
pd.options.display.max_columns = 30

In [3]:
raw_train = pd.read_csv("train.csv")

In [4]:
raw_train.shape

(69713, 22)

In [5]:
raw_train.head()

,ID,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved
0,APPC90493171225,Female,23/07/79,15/07/16,C10001,A,COM0044082,A,4.0,2000.0,B001,P,N,S122,G,0.0,NaN,NaN,NaN,NaN,0,0
1,APPD40611263344,Male,07/12/86,04/07/16,C10003,A,COM0000002,C,1.0,3500.0,B002,P,Y,S122,G,0.0,20000.0,2.0,13.25,953.0,10,0
2,APPE70289249423,Male,10/12/82,19/07/16,C10125,C,COM0005267,C,4.0,2250.0,B003,G,Y,S143,B,0.0,45000.0,4.0,NaN,NaN,0,0
3,APPF80273865537,Male,30/01/89,09/07/16,C10477,C,COM0004143,A,4.0,3500.0,B003,G,Y,S143,B,0.0,92000.0,5.0,NaN,NaN,7,0
4,APPG60994436641,Male,19/04/85,20/07/16,C10002,A,COM0001781,A,4.0,10000.0,B001,P,Y,S134,B,2500.0,50000.0,2.0,NaN,NaN,10,0


## First observations 

* A lot of categorical variables 
* Will need to turn DOB into date time -> extract years/ 2017-year as new feature 
* Lead creation date -- might need to turn that into number of days since lead generation 
* Will need to find out how many categories in several groups 
* Some variables has NaN's -- have have to impute mean/median 

In [6]:
raw_train.describe()

,Employer_Category2,Monthly_Income,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved
count,65415.000000,6.971300e+04,69662.000000,42004.000000,42004.000000,22276.000000,22276.000000,69713.000000,69713.000000
mean,3.720187,5.622283e+03,360.928751,39429.982859,3.890629,19.213570,1101.466242,3.948446,0.014631
std,0.807374,1.747671e+05,2288.517927,30727.595990,1.167491,5.847136,752.661394,3.819214,0.120073
min,1.000000,0.000000e+00,0.000000,5000.000000,1.000000,11.990000,118.000000,0.000000,0.000000
25%,4.000000,1.650000e+03,0.000000,20000.000000,3.000000,15.250000,649.000000,0.000000,0.000000
50%,4.000000,2.500000e+03,0.000000,30000.000000,4.000000,18.000000,941.000000,2.000000,0.000000
75%,4.000000,4.000000e+03,350.000000,50000.000000,5.000000,20.000000,1295.000000,7.000000,0.000000
max,4.000000,3.838384e+07,545436.500000,300000.000000,6.000000,37.000000,13556.000000,10.000000,1.000000


I guess describe takes the continous ones and we can see which have more values associated with them  
A new analysis might see which variables are just positive with the approved target variable

In [7]:
raw_train.index = raw_train["ID"]
raw_train.drop(["ID"], axis = 1, inplace = True)

In [8]:
# This date time step takes some time for some reason
raw_train["DOB"] = pd.to_datetime(raw_train["DOB"])
raw_train["Lead_Creation_Date"] = pd.to_datetime(raw_train["Lead_Creation_Date"])

In [9]:
raw_train["DOB_year"] = raw_train["DOB"].apply(lambda x: x.year)
raw_train["age_DOB"] = 2018 - raw_train["DOB_year"]

In [10]:
todays_date = pd.to_datetime("19/01/2017")
raw_train["Days_since_lead_creation"] = raw_train["Lead_Creation_Date"].apply(lambda x: (todays_date - x).days)

In [11]:
raw_train.drop(["DOB", "Lead_Creation_Date"], axis = 1, inplace = True)

In [12]:
raw_train.columns

Index(['Gender', 'City_Code', 'City_Category', 'Employer_Code',
       'Employer_Category1', 'Employer_Category2', 'Monthly_Income',
       'Customer_Existing_Primary_Bank_Code', 'Primary_Bank_Type', 'Contacted',
       'Source', 'Source_Category', 'Existing_EMI', 'Loan_Amount',
       'Loan_Period', 'Interest_Rate', 'EMI', 'Var1', 'Approved', 'DOB_year',
       'age_DOB', 'Days_since_lead_creation'],
      dtype='object')

### Let's start looking at those categorical columns 

In [13]:
# Assuming dataframe.describe() only take the numeric values... 
cat_cols = [col for col in raw_train.columns.tolist() if col not in raw_train.describe().columns.tolist()]
cat_cols

['Gender',
 'City_Code',
 'City_Category',
 'Employer_Code',
 'Employer_Category1',
 'Customer_Existing_Primary_Bank_Code',
 'Primary_Bank_Type',
 'Contacted',
 'Source',
 'Source_Category']

In [14]:
numb_unique = {col:(raw_train[col].nunique()) for col in cat_cols}
#print(numb_unique)
numb_unique

{'City_Category': 3,
 'City_Code': 678,
 'Contacted': 2,
 'Customer_Existing_Primary_Bank_Code': 57,
 'Employer_Category1': 3,
 'Employer_Code': 36617,
 'Gender': 2,
 'Primary_Bank_Type': 2,
 'Source': 29,
 'Source_Category': 7}

Employer_Code and city code might create too many one hot variables  
Let's drop those two variables and then get the dummies for the other ones 

In [15]:
cats_to_drop = ["City_Code", "Employer_Code"]
raw_train.drop(cats_to_drop, axis = 1, inplace=True)

In [16]:
for col in cats_to_drop:
    if col in numb_unique.keys():
        numb_unique.pop(col)
        print("Popped ", col, " from dict")

Popped  City_Code  from dict
Popped  Employer_Code  from dict


In [17]:
[key for key in numb_unique.keys()]

['Primary_Bank_Type',
 'Contacted',
 'Source_Category',
 'Customer_Existing_Primary_Bank_Code',
 'City_Category',
 'Source',
 'Employer_Category1',
 'Gender']

In [18]:
numb_unique

{'City_Category': 3,
 'Contacted': 2,
 'Customer_Existing_Primary_Bank_Code': 57,
 'Employer_Category1': 3,
 'Gender': 2,
 'Primary_Bank_Type': 2,
 'Source': 29,
 'Source_Category': 7}

In [19]:
# Append dummy variables at to the main dataset 
for col in numb_unique.keys():
    temp_dummies = pd.get_dummies(raw_train[col], prefix=col)
    raw_train = raw_train.join(temp_dummies)

In [20]:
dummies = pd.DataFrame(index = raw_train.index)
for col in numb_unique.keys():
    temp_dummies = pd.get_dummies(raw_train[col], prefix=col)
    dummies = dummies.join(temp_dummies)
#raw_train.join(dummies)
dummies = dummies.join(raw_train["Approved"])

In [21]:
dummies[dummies["Approved"] == 1].describe()

,Primary_Bank_Type_G,Primary_Bank_Type_P,Contacted_N,Contacted_Y,Source_Category_A,Source_Category_B,Source_Category_C,Source_Category_D,Source_Category_E,Source_Category_F,Source_Category_G,Customer_Existing_Primary_Bank_Code_B001,Customer_Existing_Primary_Bank_Code_B002,Customer_Existing_Primary_Bank_Code_B003,Customer_Existing_Primary_Bank_Code_B004,...,Source_S154,Source_S155,Source_S156,Source_S157,Source_S158,Source_S159,Source_S160,Source_S161,Source_S162,Employer_Category1_A,Employer_Category1_B,Employer_Category1_C,Gender_Female,Gender_Male,Approved
count,1020.000000,1020.000000,1020.000000,1020.000000,1020.0,1020.000000,1020.000000,1020.0,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,...,1020.0,1020.0,1020.0,1020.000000,1020.000000,1020.000000,1020.0,1020.000000,1020.0,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.0
mean,0.192157,0.788235,0.169608,0.830392,0.0,0.485294,0.137255,0.0,0.014706,0.005882,0.356863,0.297059,0.236275,0.084314,0.114706,...,0.0,0.0,0.0,0.002941,0.000980,0.030392,0.0,0.001961,0.0,0.353922,0.345098,0.296078,0.243137,0.756863,1.0
std,0.394189,0.408759,0.375472,0.375472,0.0,0.500029,0.344285,0.0,0.120432,0.076508,0.479309,0.457187,0.425001,0.277994,0.318823,...,0.0,0.0,0.0,0.054179,0.031311,0.171748,0.0,0.044259,0.0,0.478420,0.475633,0.456750,0.429188,0.429188,0.0
min,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,0.000000,1.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,1.0
50%,0.000000,1.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,1.0
75%,0.000000,1.000000,0.000000,1.000000,0.0,1.000000,0.000000,0.0,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.000000,1.000000,1.000000,0.000000,1.000000,1.0
max,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.0,0.0,0.0,1.000000,1.000000,1.000000,0.0,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.0


In [22]:
raw_train.drop(numb_unique.keys(), axis = 1, inplace=True)

In [23]:
len(raw_train.columns)

117

## Replacate for Test Set 

In [24]:
raw_test = pd.read_csv("test.csv")

In [25]:
raw_test.shape

(30037, 21)

In [26]:
raw_test.head()

,ID,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1
0,APPA70109647212,Male,03/06/88,05/07/16,C10028,C,COM0002222,A,4.0,2150.0,B002,P,Y,S122,B,0.0,10000.0,3.0,20.0,372.0,4
1,APPB10687939341,Male,13/07/81,01/07/16,C10003,A,COM0001784,C,4.0,4200.0,B004,P,Y,S133,C,0.0,69000.0,5.0,24.0,1985.0,7
2,APPC80449411414,Female,19/11/90,01/07/16,C10009,B,COM0045260,B,4.0,1000.0,B002,P,N,S133,B,0.0,NaN,NaN,NaN,NaN,0
3,APPD30665094501,Female,15/10/92,01/07/16,C10005,A,COM0000085,A,3.0,1465.0,B001,P,N,S133,C,0.0,NaN,NaN,NaN,NaN,0
4,APPE80379821637,Male,21/09/88,01/07/16,C10005,A,COM0006422,A,4.0,2340.0,B004,P,Y,S143,B,500.0,10000.0,2.0,NaN,NaN,0


In [27]:
raw_test.index = raw_test["ID"]
raw_test.drop(["ID"], axis = 1, inplace = True)

In [28]:
# Date time variable s
raw_test["DOB"] = pd.to_datetime(raw_test["DOB"])
raw_test["Lead_Creation_Date"] = pd.to_datetime(raw_test["Lead_Creation_Date"])
raw_test["DOB_year"] = raw_test["DOB"].apply(lambda x: x.year)
raw_test["age_DOB"] = 2018 - raw_test["DOB_year"]
todays_date = pd.to_datetime("19/01/2017")
raw_test["Days_since_lead_creation"] = raw_test["Lead_Creation_Date"].apply(lambda x: (todays_date - x).days)
raw_test.drop(["DOB", "Lead_Creation_Date"], axis = 1, inplace = True)

In [29]:
raw_test.drop(cats_to_drop, axis = 1, inplace = True)
for col in numb_unique.keys():
    temp_dummies = pd.get_dummies(raw_test[col], prefix=col)
    raw_test = raw_test.join(temp_dummies)
raw_test.drop(numb_unique.keys(), axis = 1, inplace=True)

In [30]:
len(raw_test.columns)

114

In [31]:
# Columns in train that are not in test 
rem_cols_train = [col for col in raw_train.columns.tolist() if col not in raw_test.columns.tolist() and col != "Approved" ]
rem_cols_train

['Customer_Existing_Primary_Bank_Code_B056',
 'Source_S130',
 'Source_S135',
 'Source_S140',
 'Source_S154',
 'Source_S160']

In [32]:
# Columns that are in test but not in train 
rem_cols_test = [col for col in raw_test.columns.tolist() if col not in raw_train.columns.tolist() and col != "Approved" ]
rem_cols_test

['Source_S126', 'Source_S131', 'Source_S132', 'Source_S142']

In [33]:
raw_train.drop(rem_cols_train, axis = 1, inplace=True)
raw_test.drop(rem_cols_test, axis = 1, inplace=True)
raw_train.to_csv("Train_Modeling_Dataset.csv")
raw_test.to_csv("Test_Modeling_Dataset.csv")

In [34]:
"Approved" in raw_train.columns

True

In [35]:
len(raw_test.columns) == len(raw_train.columns) -1 # 1 for approved (dependent variable )

True

In [36]:
len(raw_train.columns)

111